Imports

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import ee
import cdsapi
import xarray as xr
import json
from shapely.geometry import Polygon, mapping, box
import zipfile
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import concurrent.futures
import contextlib
import rasterio
from rasterio.warp import reproject, Resampling
# from osgeo import gdal
import sys
paths_to_add = ["/homes/vk223/ProjectFlood", "/home/vkhandekar/project_flood"]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/vkhandekar/project_flood/.venv/lib/python3.10/site-packages', '/homes/vk223/ProjectFlood', '/home/vkhandekar/project_flood']


In [3]:
ee.Authenticate()
ee.Initialize(project='ee-varunkhandekar')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


Flood Helpers

In [2]:
def generate_overlapping_events(file_path: str):
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path, engine='openpyxl')
    else:
        raise ValueError("Please provide a CSV or Excel file.")
    
    # Data file integrity checks
    required_columns = ['index', 'dfo_began_uk', 'dfo_ended_uk']
    assert all(column in df.columns for column in required_columns), f"Missing columns: {[column for column in required_columns if column not in df.columns]}"

    df['dfo_began_uk'] = pd.to_datetime(df['dfo_began_uk'], dayfirst=True)
    df['dfo_ended_uk'] = pd.to_datetime(df['dfo_ended_uk'], dayfirst=True)

    df['overlapping_events'] = ''

    for i, row in df.iterrows():
        overlaps = []
        for j, compare_row in df.iterrows():
            if i != j:
                if not (row['dfo_ended_uk'] < compare_row['dfo_began_uk'] or row['dfo_began_uk'] > compare_row['dfo_ended_uk']):
                    overlaps.append(j)
        df.at[i, 'overlapping_events'] = pd.NA
        if len(overlaps) != 0:
            df.at[i, 'overlapping_events'] = ','.join(map(str, overlaps))

    return df

df = generate_overlapping_events('Bangladesh_Flood_Events.xlsx')
df.head()
len(df.index)


103

In [16]:
def generate_flood_events(file_path: str):
    # Preprocess raw flood data
    df = generate_overlapping_events(file_path)
    
    overlapping_dict = {}

    for idx, row in df.iterrows():
        dfo_began_uk = row['dfo_began_uk']
        overlapping_events = row['overlapping_events']
        
        # Create list for overlapping events (list includes the current row itself)
        overlapping_events_list = [idx] # add current row to list
        if pd.notna(overlapping_events):
            overlapping_events_list = [i for i in list(map(int, overlapping_events.split(',')))] # get rows of overlapping events
        
        overlapping_events_list = [df['index'][i] for i in overlapping_events_list]

        # Find earliest event date; 
        # REMOVE THIS LOGIC IF YOU WANT MORE DATAPOINTS, I.E. NOT MERGING TO THE START DATE OF THE EARLIEST OVERLAPPING EVENT
        earliest_event_date = dfo_began_uk
        for event in overlapping_events_list:
            event_date = df[df['index'] == event]['dfo_began_uk'].values[0]
            event_date = pd.Timestamp(event_date)
            if event_date < earliest_event_date:
                earliest_event_date = event_date
        
        # Add the events to the dictionary with the earliest event date as key
        if earliest_event_date not in overlapping_dict:
            overlapping_dict[earliest_event_date] = overlapping_events_list
        else:
            overlapping_dict[earliest_event_date].extend(overlapping_events_list)
            overlapping_dict[earliest_event_date] = list(set(overlapping_dict[earliest_event_date]))
    
    return overlapping_dict


overlapping_dict = generate_flood_events('Bangladesh_Flood_Events.xlsx')
soil_moisture_date = pd.to_datetime(list(overlapping_dict.keys())[1])- timedelta(days=1)
print(soil_moisture_date)
print(len(overlapping_dict.keys()))


2001-07-07 00:00:00
56


In [14]:
shapefile_path = 'bangladesh-outline_68.geojson'
# 'bangladesh-detailed-boundary_858.geojson' 'bangladesh-outline_68.geojson'


with open(shapefile_path, 'r') as file:
    geojson_data = json.load(file)

# Extract polygon coordinates
polygon_coordinates = []

for feature in geojson_data['features']:
    if feature['geometry']['type'] == 'Polygon':
        polygon_coordinates.append(feature['geometry']['coordinates'])
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            polygon_coordinates.append(polygon)

ee_geometry = ee.Geometry.MultiPolygon(polygon_coordinates)
# Create the polygon
polygon = Polygon(polygon_coordinates[0][0])

# print(ee_geometry.getInfo()['coordinates'])
# print(polygon_coordinates[0])
# print(ee_geometry.getInfo()['coordinates']==polygon_coordinates[0])
print(polygon.bounds)
print(ee_geometry.bounds().getInfo())
bbox_ee = ee.Geometry.BBox(*polygon.bounds)

bbox = box(*polygon.bounds)

geojson = [mapping(bbox)]
geojson

(88.084422, 20.670883, 92.672721, 26.446526)
{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[88.084422, 20.670882999999975], [92.672721, 20.670882999999975], [92.672721, 26.446526000000024], [88.084422, 26.446526000000024], [88.084422, 20.670882999999975]]]}


[{'type': 'Polygon',
  'coordinates': (((92.672721, 20.670883),
    (92.672721, 26.446526),
    (88.084422, 26.446526),
    (88.084422, 20.670883),
    (92.672721, 20.670883)),)}]

In [18]:

era5_soil_moisture = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
soil_moisture = ee.Image(era5_soil_moisture.filterDate(soil_moisture_date.strftime(r"%Y-%m-%d"), 
                                                        (soil_moisture_date+pd.Timedelta(days=1)).strftime(r'%Y-%m-%d')).first())
soil_moisture = soil_moisture.select("volumetric_soil_water_layer_1")
soil_moisture = soil_moisture.unmask(1)
# soil_moisture = soil_moisture.resample(mode="bilinear")
export_task = ee.batch.Export.image.toDrive(
    image=soil_moisture,
    description=f'BangladeshSoilMoisture{soil_moisture_date.year}{soil_moisture_date.month:02}{soil_moisture_date.day:02}',
    folder='BangladeshSoilMoisture',
    fileNamePrefix=f'BangladeshSoilMoisture{soil_moisture_date.year}{soil_moisture_date.month:02}{soil_moisture_date.day:02}',
    region=bbox_ee,
    crs='EPSG:4326',
    # scale=250,  # Adjust the scale as needed
    crsTransform=[0.002245788210298804, 0.0, 88.08430518433968, 0.0, -0.002245788210298804, 26.44864775268901],
    maxPixels=1e13 
)
export_task.start()


In [33]:
export_task.status()

{'state': 'READY',
 'description': 'BangladeshSoilMoisture20010707',
 'priority': 100,
 'creation_timestamp_ms': 1720008450010,
 'update_timestamp_ms': 1720008450010,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '3PIBKIK3GO5FEQ67SARMUKJ3',
 'name': 'projects/ee-varunkhandekar/operations/3PIBKIK3GO5FEQ67SARMUKJ3'}

Shape Helpers

In [13]:
def generate_country_outline(shapefile_path: str):
    with open(shapefile_path, 'r') as file:
        geojson_data = json.load(file)

    # Extract polygon coordinates
    polygon_coordinates = []

    for feature in geojson_data['features']:
        if feature['geometry']['type'] == 'Polygon':
            polygon_coordinates.append(feature['geometry']['coordinates'])
        elif feature['geometry']['type'] == 'MultiPolygon':
            for polygon in feature['geometry']['coordinates']:
                polygon_coordinates.append(polygon)
    
    return Polygon(polygon_coordinates[0][0])

bangladesh_shape = generate_country_outline('bangladesh-outline_68.geojson')
bangladesh_bounding_box = box(*bangladesh_shape.bounds)
bangladesh_bounding_box_ee = ee.Geometry.BBox(*bangladesh_shape.bounds)

Soil Moisture Helpers

In [5]:
def pull_soil_moisture(soil_moisture_date, output_path: str, shape: Polygon):
    # Set up Copernicus
    c = cdsapi.Client()

    zip_path = f'data/soil_moisture_bangladesh_{soil_moisture_date.year}{soil_moisture_date.month:02}{soil_moisture_date.day:02}.zip'
    # with contextlib.redirect_stdout(None):
    c.retrieve(
    'satellite-soil-moisture',
    {
        'format': 'zip',
        'year': f'{soil_moisture_date.year}',
        'day': f'{soil_moisture_date.day:02}',
        'variable': 'surface_soil_moisture',
        'type_of_sensor': 'active',
        'time_aggregation': 'day_average',
        'month': f'{soil_moisture_date.month:02}',
        'type_of_record': 'cdr', #may need to change to icdr when considering going 'live'
        'version': 'v202212',
    },
    zip_path
    )

    new_path = os.path.join(output_path, f'soil_moisture_bangladesh_{soil_moisture_date.year}{soil_moisture_date.month:02}{soil_moisture_date.day:02}.nc')
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        zip_ref.extract(file_list[0], path=output_path)       
        os.rename(os.path.join(output_path, file_list[0]), new_path)
        print(f'File extracted and renamed to: {new_path}')
    os.remove(zip_path)

    # Clip the data and save down
    geojson = [mapping(shape)] # Use GeoJSON of the shape
    with xr.open_dataset(new_path, engine="netcdf4") as data:
        data = data['sm']
        data.rio.set_spatial_dims('lon', 'lat', inplace=True)
        data.rio.write_crs("EPSG:4326", inplace=True)
        
        data_clipped = data.rio.clip(geojson, all_touched=True)
    data_clipped.to_netcdf(new_path, mode='w')

    return new_path
    

In [28]:
def authenticate(config_file_path: str):
    """Authenticate using PyDrive and return a GoogleDrive object."""
    with open(config_file_path) as config_file:
        config = json.load(config_file)
    google_drive_credentials_path = config['google_drive_credentials']
    google_drive_oauth_path = config['google_drive_oauth']

    gauth = GoogleAuth()
    gauth.LoadClientConfigFile(google_drive_credentials_path)

    # Try to load saved credentials
    gauth.LoadCredentialsFile(google_drive_oauth_path)
    if gauth.credentials is None:
        try:
            gauth.LocalWebserverAuth()  # Creates local webserver and auto handles authentication
        except:
            gauth.CommandLineAuth()  # Use this if LocalWebserverAuth fails
        gauth.SaveCredentialsFile(google_drive_oauth_path)
    elif gauth.access_token_expired:
        gauth.Refresh()
        gauth.SaveCredentialsFile(google_drive_oauth_path)
    else:
        gauth.Authorize()

    return GoogleDrive(gauth)



None
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=629975352601-slkn7n0p99o6udme02pkj26fa9vvnb28.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [7]:
def send_to_google_drive(drive: GoogleDrive, file_path: str, config_file_path: str, target_location: str, overwrite=False):
    # Get config details
    with open(config_file_path) as config_file:
        config = json.load(config_file)
    folder_id = config[target_location]

    # drive = authenticate(config_file_path)
    file_name = os.path.basename(file_path)

    if overwrite:
        # Search for the file with the same name in the target folder; delete if it exists
        file_list = drive.ListFile({'q': f"title = '{file_name}' and '{folder_id}' in parents and trashed=false"}).GetList()
        for file in file_list:
            file.Delete()

    file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}]})
    file.SetContentFile(file_path)
    file.Upload()
    file = None
    

Rainfall Helpers

In [3]:
def generate_timestamps(date: pd.Timestamp, days_before: int, days_after: int):
    start_time = date - pd.Timedelta(days=days_before)
    end_time = date + pd.Timedelta(days=days_after)

    timestamps = pd.date_range(start=start_time, end=end_time, freq='3h')
    # don't include final forecast as it takes us into day t+days_after+1
    return timestamps.tolist()[:-1]

In [32]:
def convert_date_to_MSWEP_file_name(timestamp: pd.Timestamp):
    return f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.nc"

def generate_files_of_interest(drive: GoogleDrive, timestamps: list, config: str, folder: str):
    with open("config.json") as config_file:
        config = json.load(config_file)
    folder_id = config[folder]

    file_names = [convert_date_to_MSWEP_file_name(timestamp) for timestamp in timestamps]

    files = []
    for file_name in file_names:
        query = f"title='{file_name}' and '{folder_id}' in parents"
        file_list = drive.ListFile({'q': query}).GetList()
        files.extend(file_list)
    return files


ee.Authenticate()
with open('config.json') as config_file:
    config = json.load(config_file)
earth_engine_project_name = config['earth_engine_project']
ee.Initialize(project=earth_engine_project_name)
print(ee.String('Hello from the Earth Engine servers!').getInfo())


perm_water_threshold = 50 # x% of the time the area is covered in water


drive = authenticate('config.json')
query = f"title='2007174.06.nc' and '1DVR90Ud1C444bTOPeENX-3I7tgqPLnao' in parents"
file_list = drive.ListFile({'q': query}).GetList()
file_list[0]


Hello from the Earth Engine servers!


GoogleDriveFile({'kind': 'drive#file', 'userPermission': {'id': 'me', 'type': 'user', 'role': 'reader', 'kind': 'drive#permission', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1RPdc1H40PgGrySKGiS7NEgjUGsa87KQQ/permissions/me', 'etag': '"z1iyNVncsY5NbIAaVMiZUu299K8"', 'pendingOwner': False}, 'fileExtension': 'nc', 'md5Checksum': '9a154d72b00b9effa8419f07238fcf0e', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1RPdc1H40PgGrySKGiS7NEgjUGsa87KQQ', 'ownerNames': ['gloh2o.org'], 'lastModifyingUserName': 'gloh2o.org', 'editable': False, 'writersCanShare': True, 'downloadUrl': 'https://www.googleapis.com/drive/v2/files/1RPdc1H40PgGrySKGiS7NEgjUGsa87KQQ?alt=media&source=downloadUrl', 'mimeType': 'application/x-netcdf', 'parents': [{'selfLink': 'https://www.googleapis.com/drive/v2/files/1RPdc1H40PgGrySKGiS7NEgjUGsa87KQQ/parents/1DVR90Ud1C444bTOPeENX-3I7tgqPLnao', 'id': '1DVR90Ud1C444bTOPeENX-3I7tgqPLnao', 'isRoot': False, 'kind': 'drive#parentReference', 'parentLink': 'https:

In [33]:
timestamp_str = "2007174.06"

# Extract the components from the string
year = int(timestamp_str[:4])
day_of_year = int(timestamp_str[4:7])
hour = int(float(timestamp_str[7:]) * 100)

# Construct the timestamp
timestamp = pd.Timestamp(year=year, month=1, day=1) + pd.to_timedelta(f'{day_of_year - 1} days') + pd.to_timedelta(f'{hour} hours')
timestamp

Timestamp('2007-06-23 06:00:00')

In [44]:
temp_output_path = "./data/temp/"
new_file = drive.CreateFile({'id': file_list[0]['id']})
new_path = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.nc")
temp_path = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_tmp.nc")
new_file.GetContentFile(new_path)

# Clip the data and save down
geojson = [mapping(bangladesh_bounding_box)] # Use GeoJSON of the shape
with xr.open_dataset(new_path, engine="netcdf4") as data:
    data = data['precipitation']
    data.rio.set_spatial_dims('lon', 'lat', inplace=True)
    data.rio.write_crs("EPSG:4326", inplace=True)
    
    data_clipped = data.rio.clip(geojson, all_touched=True)
    data_clipped = ((data_clipped*1000)//1).astype(np.uint16)
data_clipped.to_netcdf(temp_path, mode='w')
os.remove(new_path)
os.rename(temp_path, new_path)


In [47]:

data_clipped.min()


<xarray.DataArray 'precipitation' ()> Size: 2B
array(0, dtype=uint16)
Coordinates:
    spatial_ref  int64 8B 0

In [49]:

# Now convert to .tif using GDAL
lowres_tif_file = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_lowres.tif")
precipitation_data = gdal.Open(f'NETCDF:"{new_path}":precipitation')
driver = gdal.GetDriverByName('GTiff')

output_dataset = driver.CreateCopy(lowres_tif_file, precipitation_data, 0)
output_dataset.SetProjection('EPSG:4326')

precipitation_data = None
output_dataset = None

# os.remove(new_path)



In [50]:
def reproject_and_upsample(input_file: str, output_file: str, config_file: str):
    with rasterio.open(input_file) as src:
        src_transform = src.transform
        data = src.read()
        src_crs = src.crs
        with open("config.json") as config_file:
            config = json.load(config_file)
        master_file = config['rainfall_reprojection_master']
        # Open target file to get target dimensions and transform
        with rasterio.open(master_file) as target:
            target_transform = target.transform
            target_width = target.width
            target_height = target.height
            target_crs = target.crs
            
            # Update metadata for the new file, including LZW compression
            kwargs = src.meta
            kwargs.update({
                'height': target_height,
                'width': target_width,
                'transform': target_transform,
                'crs': target_crs,
                'compress': 'lzw'
            })
        
        # Write to output
        with rasterio.open(output_file, 'w', **kwargs) as dst:
            for i, band in enumerate(data, 1): # Go through bands
                dest = np.zeros((target_height, target_width), dtype=np.float32)
                
                reproject(
                    source=band,
                    destination=dest,
                    src_transform=src_transform,
                    src_crs=src_crs,
                    dst_transform=target_transform,
                    dst_crs=target_crs,
                    resampling=Resampling.bilinear
                )

                dst.write(dest, indexes=i)

    return output_file

In [58]:
from PIL import Image
# Do reprojection, upsampling and compression before saving down

output_tif_file = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_test.tif")


lowres_image = Image.open(lowres_tif_file)
print(lowres_image.height)
with open("config.json") as config_file:
    config = json.load(config_file)
master_file = config['rainfall_reprojection_master']
target_image = Image.open(master_file)
target_width = target_image.width
target_height = target_image.height
print(lowres_image.mode)
upsampled_image = lowres_image.resize((target_width, target_height))
upsampled_image.save(output_tif_file, compression='tiff_deflate')


59
I;16


In [68]:

with rasterio.open(lowres_tif_file) as src:
    print(src.meta)
    data = src.read()
    print(data.shape)
    # with rasterio.open(master_file) as target:
    #     target_transform = target.transform
    #     target_width = target.width
    #     target_height = target.height
    #     target_crs = target.crs
        
    #     kwargs = src.meta
    #     kwargs.update({
    #         'height': target_height,
    #         'width': target_width,
    #         'transform': target_transform,
    #         'crs': target_crs,
    #     })
    # # Write to output
    # with rasterio.open(os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_test_withmeta.tif"), 'w', **kwargs) as dst:
    #     dst.write(data[0], indexes=1)

# reproject_and_upsample(lowres_tif_file, output_tif_file, config_file) #2044x2573
# os.remove(lowres_tif_file)

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': 65535.0, 'width': 47, 'height': 59, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.10000013268512228, 0.0, 88.00000298541525,
       0.0, -0.10000005261651401, 26.50000078924771)}
(1, 59, 47)


In [70]:
def remove_metadata(image_path: str):
    image = Image.open(image_path)
    data = list(image.getdata())
    image_without_metadata = Image.new(image.mode, image.size)
    image_without_metadata.putdata(data)
    image_without_metadata.save(image_path)


remove_metadata(os.path.join(temp_output_path, "2007181.21.tif"))

with rasterio.open(os.path.join(temp_output_path, "2007181.21.tif")) as src:
    print(src.meta)
    data = src.read()
    print(data.shape)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 2044, 'height': 2573, 'count': 1, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
(1, 2573, 2044)


/home/vkhandekar/project_flood/.venv/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [10]:
def pull_and_crop_rainfall_data(drive: GoogleDrive, original_file, timestamp: pd.Timestamp, temp_output_path: str, shape: Polygon, config_file: str):

    new_file = drive.CreateFile({'id': original_file['id']})
    new_path = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.nc")
    temp_path = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_tmp.nc")
    new_file.GetContentFile(new_path)

    # Clip the data and save down
    geojson = [mapping(shape)] # Use GeoJSON of the shape
    with xr.open_dataset(new_path, engine="netcdf4") as data:
        data = data['precipitation']
        data.rio.set_spatial_dims('lon', 'lat', inplace=True)
        data.rio.write_crs("EPSG:4326", inplace=True)
        
        data_clipped = data.rio.clip(geojson, all_touched=True)
    data_clipped.to_netcdf(temp_path, mode='w')
    os.remove(new_path)
    os.rename(temp_path, new_path)

    # Now convert to .tif using GDAL
    lowres_tif_file = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}_lowres.tif")
    precipitation_data = gdal.Open(f'NETCDF:"{new_path}":precipitation')
    driver = gdal.GetDriverByName('GTiff')

    output_dataset = driver.CreateCopy(lowres_tif_file, precipitation_data, 0)
    output_dataset.SetProjection('EPSG:4326')

    precipitation_data = None
    output_dataset = None

    os.remove(new_path)

    # Do reprojection, upsampling and compression before saving down
    output_tif_file = os.path.join(temp_output_path, f"{timestamp.year}{timestamp.dayofyear}.{timestamp.hour:02}.tif")
    reproject_and_upsample(lowres_tif_file, output_tif_file, config_file) #2044x2573
    os.remove(lowres_tif_file)

    return output_tif_file

Initialise and Set Up APIs and Shapes

In [11]:
# Set Up Google Earth Engine
ee.Authenticate()
with open('config.json') as config_file:
    config = json.load(config_file)
earth_engine_project_name = config['earth_engine_project']
ee.Initialize(project=earth_engine_project_name)
print(ee.String('Hello from the Earth Engine servers!').getInfo())

gfd = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
perm_water = ee.ImageCollection('JRC/GSW1_4/MonthlyRecurrence')
perm_water_threshold = 25 # x% of the time the area is covered in water

# Set Up Google Drive API
drive = authenticate("config.json")

# Get Bangladesh outline
bangaldesh_coordinates = generate_country_outline('bangladesh-outline_68.geojson')
bangladesh_shape_ee = ee.Geometry.MultiPolygon(bangaldesh_coordinates)
bangladesh_shape = Polygon(bangaldesh_coordinates[0][0])

# Convert the polygon to GeoJSON format
# geojson = [mapping(bangladesh_shape)]

# Rainfall days before and after
rainfall_days_before = 2
rainfall_days_after = 1

# Max number of threads allowed to run
max_workers = 3

Hello from the Earth Engine servers!


Pull Topology Data

In [12]:
topology = ee.Image("NASA/NASADEM_HGT/001").select("elevation")

# Replace all no data values with 0 (essentially the sea)
topology = topology.unmask(0)

export_task = ee.batch.Export.image.toDrive(
    image=topology,
    description='BangladeshTopology',
    folder='EarthEngineExports/Topology',
    fileNamePrefix='BangladeshTopology',
    region=bangladesh_shape_ee.getInfo()['coordinates'],
    scale=250,  # Adjust the scale as needed
    maxPixels=1e13  # Adjust maxPixels as needed
)
export_task.start()
# export_task.status()

{'state': 'READY',
 'description': 'BangladeshTopology',
 'priority': 100,
 'creation_timestamp_ms': 1719478938633,
 'update_timestamp_ms': 1719478938633,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': '5ACKIZ2XEDOHQN5DR2RGDOW2',
 'name': 'projects/ee-varunkhandekar/operations/5ACKIZ2XEDOHQN5DR2RGDOW2'}

Pulling Requisite Flood Event Data

In [ ]:
# Loop through refined flood events
refined_flood_events = generate_flood_events('Bangladesh_Flood_Events.xlsx')

for date, events in refined_flood_events.items():
    print("============")
    print("Working on the floods on:", date)
    # Pull and mosaic overlapping flood events, 250m resolution
    print("----", "Generating flood image", "----")
    inundation_images = []
    for event in events:
        flood_image = ee.Image(gfd.filterMetadata('id', 'equals', int(event)).first()).select(['flooded'])
        inundation_images.append(flood_image)

    # Add permanent water overlay
    water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', date.month).first())
    occurrence_band = water_overlay.select('monthly_recurrence')
    water_overlay_filtered = occurrence_band.gt(perm_water_threshold)
    water_overlay_filtered = water_overlay_filtered.rename('flooded')
    inundation_images.append(water_overlay_filtered)

    # Export combo image to gdrive
    combo = ee.ImageCollection(inundation_images).reduce(ee.Reducer.max())
    export_task = ee.batch.Export.image.toDrive(
        image=combo,
        description=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        folder='EarthEngineExports/BangladeshFloodImages',
        fileNamePrefix=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        region=bangladesh_shape_ee.getInfo()['coordinates'],
        scale=250,  # Adjust the scale as needed
        maxPixels=1e13 
    )
    export_task.start()
    export_task.status()
    print("----", "Flood image done", "----", "\n")

    # Pull soil moisture data for the day before, store to gdrive
    print("----", "Gathering soil moisture images", "----")
    soil_moisture_date = pd.to_datetime(date) - timedelta(days=1)
    soil_moisture_file = pull_soil_moisture(soil_moisture_date, 'data/soil_moisture', bangladesh_shape)
    send_to_google_drive(drive, soil_moisture_file, 'config.json', 'soil_moisture_folder_id')
    print("----", "Soil moisture image done", "----", "\n")
    

    # Pull rain data 2 days preceding, 1 day 'forecast'
    print("----", "Gathering rain images", "----")
    timestamps = generate_timestamps(date, rainfall_days_before, rainfall_days_after)
    files = generate_files_of_interest(drive, timestamps, 'config.json', 'MSWEP_Past_3hr_folder_id')

    for i, timestamp in enumerate(timestamps):
        print("Rain image ", i)
        rainfall_file = pull_and_crop_rainfall_data(drive, files[i], timestamp, "./data/temp/", bangladesh_shape)
        send_to_google_drive(drive, rainfall_file, 'config.json', 'bangladesh_rainfall_folder_id')
        
    print("----", "Rain images done", "----")


    print("============")    


In [13]:
def process_flood_event(date, events):
    # print("============")
    # print("Working on the floods on:", date)
    
    # Pull and mosaic overlapping flood events, 250m resolution
    # print("----", "Generating flood image", "----")
    inundation_images = []
    for event in events:
        flood_image = ee.Image(gfd.filterMetadata('id', 'equals', int(event)).first()).select(['flooded'])
        inundation_images.append(flood_image)

    # Add permanent water overlay
    water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', date.month).first())
    occurrence_band = water_overlay.select('monthly_recurrence')
    water_overlay_filtered = occurrence_band.gt(perm_water_threshold)
    water_overlay_filtered = water_overlay_filtered.rename('flooded')
    inundation_images.append(water_overlay_filtered)

    # Export combo image to gdrive
    combo = ee.ImageCollection(inundation_images).reduce(ee.Reducer.max())
    export_task = ee.batch.Export.image.toDrive(
        image=combo,
        description=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        folder='EarthEngineExports/BangladeshFloodImages',
        fileNamePrefix=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        region=bangladesh_shape_ee.getInfo()['coordinates'],
        scale=250,  # Adjust the scale as needed
        maxPixels=1e13 
    )
    export_task.start()
    export_task.status()
    # print("----", "Flood image done", "----", "\n")

    # Pull soil moisture data for the day before, store to gdrive
    # print("----", "Gathering soil moisture images", "----")
    soil_moisture_date = pd.to_datetime(date) - timedelta(days=1)
    soil_moisture_file = pull_soil_moisture(soil_moisture_date, 'data/soil_moisture', bangladesh_shape)
    send_to_google_drive(drive, soil_moisture_file, 'config.json', 'soil_moisture_folder_id')
    # print("----", "Soil moisture image done", "----", "\n")
    
    # Pull rain data 2 days preceding, 1 day 'forecast'
    # print("----", "Gathering rain images", "----")
    timestamps = generate_timestamps(date, rainfall_days_before, rainfall_days_after)
    files = generate_files_of_interest(drive, timestamps, 'config.json', 'MSWEP_Past_3hr_folder_id')

    for i, timestamp in enumerate(timestamps):
        # print("Rain image ", i)
        rainfall_file = pull_and_crop_rainfall_data(drive, files[i], timestamp, "./data/temp/", bangladesh_shape)
        send_to_google_drive(drive, rainfall_file, 'config.json', 'bangladesh_rainfall_folder_id')
        
    # print("----", "Rain images done", "----")
    # print("============")


In [ ]:
refined_flood_events = generate_flood_events('Bangladesh_Flood_Events.xlsx')

# Submit jobs, thread pool as I/O bound processes i.e. the API requests are the biggest limiting factor
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_flood_event, date, events) for date, events in refined_flood_events.items()]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())


# for future in concurrent.futures.as_completed(futures):
#     try:
#         future.result()
#     except Exception as exc:
#         print(f'Generated an exception: {exc}')

Pulling Requisite Non-Flood Event Data

In [ ]:
# Extract the relevant columns for flood events
# df_dates = df[['dfo_began_uk', 'dfo_ended_uk']].dropna()

def generate_random_non_flood_dates(file_path: str, num_dates: int, safety_window: int):
    # Read in data file
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        df = pd.read_excel(file_path, engine='openpyxl')
    else:
        raise ValueError("Please provide a CSV or Excel file.")

    # Data file integrity checks
    required_columns = ['dfo_began_uk', 'dfo_ended_uk']
    assert all(column in df.columns for column in required_columns), f"Missing columns: {[column for column in required_columns if column not in df.columns]}"

    df['dfo_began_uk'] = pd.to_datetime(df['dfo_began_uk'])
    df['dfo_ended_uk'] = pd.to_datetime(df['dfo_ended_uk'])

    # Define start and end dates for random date generation
    start_date = df['dfo_began_uk'].min()
    end_date = df['dfo_ended_uk'].max()

    # Define periods to avoid looking at
    exclusion_periods = list(zip(df['dfo_began_uk'], df['dfo_ended_uk']))

    random_dates = []
    while len(random_dates) < num_dates:
        random_date = start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days))
        # Make sure date is not during any of the periods to be avoided i.e. when there was a flood
        if all(not(period_start - timedelta(days=safety_window) <= random_date <= period_end + timedelta(days=safety_window)) 
               for period_start, period_end in exclusion_periods):
            random_dates.append(random_date)
    
    return random_dates

In [ ]:
num_dates_to_generate = 100
safety_window = 10
random_dates = generate_random_non_flood_dates('Bangladesh_Flood_Events.xlsx', num_dates_to_generate, safety_window)

for date in random_dates:
    print("============")
    print("Working on the non-flood event on:", date)
    # Get permanent water
    print("----", "Generating water image", "----")
    water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', date.month).first())
    occurrence_band = water_overlay.select('monthly_recurrence')
    water_overlay_filtered = occurrence_band.gt(perm_water_threshold) 
    water_overlay_filtered = water_overlay_filtered.rename('flooded')

    export_task = ee.batch.Export.image.toDrive(
        image=water_overlay_filtered,
        description=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        folder='EarthEngineExports/BangladeshNonFloodImages',
        fileNamePrefix=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        region=bangladesh_shape_ee.getInfo()['coordinates'],
        scale=250,  # Adjust the scale as needed
        maxPixels=1e13 
    )
    export_task.start()
    export_task.status()
    print("----", "Flood image done", "----", "\n")

    # Get soil moisture data
    print("----", "Gathering soil moisture images", "----")
    soil_moisture_date = pd.to_datetime(date) - timedelta(days=1)
    soil_moisture_file = pull_soil_moisture(soil_moisture_date, 'data/soil_moisture')
    send_to_google_drive(soil_moisture_file)
    print("----", "Soil moisture image done", "----", "\n")
    

    # Pull rain data 2 days preceding, 1 day 'forecast'
    print("----", "Gathering rain images", "----")
    timestamps = generate_timestamps(date, rainfall_days_before, rainfall_days_after)
    files = generate_files_of_interest(drive, timestamps, 'config.json', 'MSWEP_Past_3hr_folder_id')

    for i, timestamp in enumerate(timestamps):
        print("Rain image ", i)
        rainfall_file = pull_and_crop_rainfall_data(drive, files[i], timestamp, "./data/temp/", bangladesh_shape)
        send_to_google_drive(drive, rainfall_file, 'config.json', 'bangladesh_rainfall_folder_id')
        
    print("----", "Rain images done", "----")


    print("============")    



In [ ]:
def process_non_flood_event(date):
    # print("============")
    # print("Working on the non-flood event on:", date)
    
    # Get permanent water
    # print("----", "Generating water image", "----")
    water_overlay = ee.Image(perm_water.filterMetadata('month', 'equals', date.month).first())
    occurrence_band = water_overlay.select('monthly_recurrence')
    water_overlay_filtered = occurrence_band.gt(perm_water_threshold) 
    water_overlay_filtered = water_overlay_filtered.rename('flooded')

    export_task = ee.batch.Export.image.toDrive(
        image=water_overlay_filtered,
        description=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        folder='EarthEngineExports/BangladeshNonFloodImages',
        fileNamePrefix=f'BangladeshWater{date.year}{date.month:02}{date.day:02}',
        region=bangladesh_shape_ee.getInfo()['coordinates'],
        scale=250,  # Adjust the scale as needed
        maxPixels=1e13 
    )
    export_task.start()
    export_task.status()
    # print("----", "Flood image done", "----", "\n")

    # Get soil moisture data
    # print("----", "Gathering soil moisture images", "----")
    soil_moisture_date = pd.to_datetime(date) - timedelta(days=1)
    soil_moisture_file = pull_soil_moisture(soil_moisture_date, 'data/soil_moisture')
    send_to_google_drive(soil_moisture_file)
    # print("----", "Soil moisture image done", "----", "\n")

    # Get rain data
    # print("----", "Gathering rain images", "----")
    timestamps = generate_timestamps(date, rainfall_days_before, rainfall_days_after)
    files = generate_files_of_interest(drive, timestamps, 'config.json', 'MSWEP_Past_3hr_folder_id')

    for i, timestamp in enumerate(timestamps):
        # print("Rain image ", i)
        rainfall_file = pull_and_crop_rainfall_data(drive, files[i], timestamp, "./data/temp/", bangladesh_shape)
        send_to_google_drive(drive, rainfall_file, 'config.json', 'bangladesh_rainfall_folder_id')
    
    # print("----", "Rain images done", "----")

    # print("============")


In [ ]:
num_dates_to_generate = 100
safety_window = 10
random_dates = generate_random_non_flood_dates('Bangladesh_Flood_Events.xlsx', num_dates_to_generate, safety_window)

# Submit jobs, thread pool as I/O bound processes i.e. the API requests are the biggest limiting factor
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_non_flood_event, date) for date in random_dates]
    for future in concurrent.futures.as_completed(futures):
        print(future.result())